In [ ]:
import csv

with open('/content/drive/MyDrive/est_grad.csv', encoding='utf-8') as uflaCsv:
    reader = csv.reader(uflaCsv)
    for i, row in enumerate(reader):
        print(row)

In [ ]:
import csv

with open('novo_csv.csv', encoding='utf-8') as uflaCsv:
    reader = csv.reader(uflaCsv)
    
    # Pula a primeira linha e imprime os nomes das colunas
    header = next(reader)
    print(header)

In [ ]:
import csv
import json
import statistics
import matplotlib.pyplot
import seaborn
import pandas

colunas_indesejadas = ['registro_academico', 'ano_letivo_referencia', 'codigo_oferta_curso', 'codigo_inep', 'nome_matriz_curricular', 'sistema_origem', 'data_atualizacao_dw']

with open('novo_csv.csv', mode='w', newline='', encoding='utf-8') as novoCsv:
    writer = csv.writer(novoCsv)
    
    with open('/content/drive/MyDrive/est_grad.csv', encoding='utf-8') as uflaCsv:
        reader = csv.reader(uflaCsv)
        header = next(reader)
        colunas_desejadas = [coluna for coluna in header if coluna not in colunas_indesejadas]
        writer.writerow(colunas_desejadas)
        
        for row in reader:
            nova_row = [row[header.index(coluna)] for coluna in colunas_desejadas]
            writer.writerow(nova_row)

# Abre o arquivo CSV modificado
with open('novo_csv.csv', encoding='utf-8') as uflaCsv:
    reader = csv.DictReader(uflaCsv)

    cursos_por_ano = {}

    for row in reader:
        curso = row['curso']
        ano_ingresso = row['semestre_letivo_ingresso'].split('/')[0]

        if curso not in cursos_por_ano:
            cursos_por_ano[curso] = {}

        if ano_ingresso not in cursos_por_ano[curso]:
            cursos_por_ano[curso][ano_ingresso] = 0

        cursos_por_ano[curso][ano_ingresso] += 1

    # Cria um dicionário com os dados
    cursos = {}
    for curso in cursos_por_ano:
        anos_matriculas = {}
        for ano in cursos_por_ano[curso]:
            anos_matriculas[int(ano)] = cursos_por_ano[curso][ano]

        media = sum(anos_matriculas.values())/len(anos_matriculas)
        mediana = statistics.median(anos_matriculas.values())
        if len(anos_matriculas) > 1:
          desvio_padrao = statistics.stdev(anos_matriculas.values())
        else:
          desvio_padrao = 0

        cursos[curso] = {
            'media': round(media, 2),
            'mediana': mediana,
            'desvio_padrao': round(desvio_padrao, 2),
            'anos_matriculas': anos_matriculas
        }

    # Salva os dados em um arquivo JSON
    with open('dados_cursos.json', 'w', encoding='utf-8') as f:
        json.dump(cursos, f, ensure_ascii=False, indent=4)

# Abre o arquivo CSV modificado
with open('novo_csv.csv', encoding='utf-8') as uflaCsv:
    reader = csv.DictReader(uflaCsv)

    cursos_por_ano = {}

    for row in reader:
        curso = row['curso']
        ano_ingresso = row['semestre_letivo_ingresso'].split('/')[0]

        if curso not in cursos_por_ano:
            cursos_por_ano[curso] = []

        cursos_por_ano[curso].append(int(ano_ingresso))

# Percorre todos os cursos e cria um gráfico de distribuição e boxplot para cada um
for curso in cursos_por_ano:
    # Cria um DataFrame com as informações do curso atual
    df = pandas.DataFrame(cursos_por_ano[curso], columns=['Ano de ingresso'])

    # Cria o gráfico de distribuição do curso atual
    fig, ax = matplotlib.pyplot.subplots(figsize=(12, 6))
    seaborn.histplot(data=df, x='Ano de ingresso', ax=ax)
    ax.set_title(f'Distribuição de alunos por ano para o curso de {curso}')
    matplotlib.pyplot.show()

    # Cria o boxplot do curso atual
    fig, ax = matplotlib.pyplot.subplots(figsize=(12, 6))
    seaborn.boxplot(data=df, x='Ano de ingresso', ax=ax)
    ax.set_title(f'Boxplot de ingressões para o curso de {curso}')
    matplotlib.pyplot.show()